In [103]:
import pandas as pd
from datetime import datetime
import requests
import streamlit as st


API_KEY = st.secrets["INTAKEQ_API_KEY"]
BASE = "https://intakeq.com/api/v1/"

def fetch_appointments_scheduled_between(start_date, end_date):
    """
    Fetch appointments scheduled between start_date and end_date
    
    https://support.intakeq.com/article/204-intakeq-appointments-api
    """
    base_url = BASE + 'appointments'
    # create a session
    session = requests.Session()
    # add api key to headers
    session.headers.update({
        "X-Auth-Key": API_KEY,
        "User-Agent": "IntakeQ-API-Client/1.0"
    })
    # make request
    response = session.get(
        f"{base_url}?&startDate={start_date}&endDate={end_date}&deletedOnly=false"
        )
    # close session
    session.close()
    return response.json()

def update_appointments_data():
    # load existing data
    df = pd.read_csv('/Users/davidsamuel/Projects/cumulative_onboarding/data/appointments.csv')
    #last updated
    last_updated = pd.to_datetime(df['DateCreated'], unit='ms').sort_values().max()
    if last_updated.date() == datetime.now().date():
        print("Appointment data is already up to date.")
    else:
        # create start date from most recent appointment DateCreated
        start_date = (last_updated - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = datetime.now().date().strftime('%Y-%m-%d')

        print(f"Fetching appointments updated since {start_date} to {end_date}")
        response = fetch_appointments_scheduled_between(start_date, end_date)
        new_data = pd.DataFrame(response)
        if not new_data.empty:
            # Keep only rows in new_data whose 'Id' is not already in df
            new_unique = new_data[~new_data['Id'].isin(df['Id'])]
            print(f"{len(new_unique)} new appointments")

            # Concatenate
            new_appts = pd.concat([df, new_unique], ignore_index=True)
        else:
            print("No new appointments found.")


def fetch_clients_created_between(start_date, end_date):
    """
    Fetch clients created between start_date and end_date

    https://support.intakeq.com/article/251-intakeq-client-api
    """
    base_url = BASE + 'clients'
    # create a session
    session = requests.Session()
    # add api key to headers
    session.headers.update({
        "X-Auth-Key": API_KEY,
        "User-Agent": "IntakeQ-API-Client/1.0"
    })
    # make request
    response = session.get(
        f"{base_url}?&dateCreatedStart={start_date}&dateCreatedEnd={end_date}&deletedOnly=false&includeProfile=true"
        )
    # close session
    session.close()
    return response.json()

def update_clients_data():
    # load existing data
    df = pd.read_csv('/Users/davidsamuel/Projects/cumulative_onboarding/data/clients.csv')
    # check most recent date created
    last_updated = pd.to_datetime(df['DateCreated'], unit='ms').sort_values().max()
    if last_updated.date() == datetime.now().date():
        print("Client data is already up to date.")
    else:
        # create start date from most recent appointment DateCreated
        start_date = (pd.to_datetime(df['DateCreated'], unit='ms').sort_values().max() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = datetime.now().date().strftime('%Y-%m-%d')

        print(f"Fetching clients created between {start_date} and {end_date}")
        response = fetch_clients_created_between(start_date, end_date)
        new_data = pd.DataFrame(response)
        if not new_data.empty:
            # Keep only rows in new_data whose 'ClientId' is not already in df
            new_unique = new_data[~new_data['ClientId'].isin(df['ClientId'])]
            print(f"{len(new_unique)} new clients")

            # Concatenate
            new_clients = pd.concat([df, new_unique], ignore_index=True)
        else:
            print("No new clients found.")

In [104]:
update_appointments_data()
update_clients_data()

Appointment data is already up to date.
Client data is already up to date.


In [9]:
import pandas as pd
appts = pd.read_csv('data/appt_dates.csv')
clients = pd.read_csv('data/dates.csv')

In [9]:
clients.sort_values('DateCreated', ascending=False).tail()

,DateCreated
58,01/07/2025
128,01/05/2024
144,01/05/2022
271,01/04/2023
28,01/03/2025


In [13]:
appointments = pd.read_csv('data/appointments.csv')
clients = pd.read_csv('data/clients.csv')

In [14]:
appointments['ServiceId'].value_counts()

ServiceId
1efe2465-4741-48c8-8408-114818cdce74    2482
64689f67-0ba6-4b73-b5ed-2d42db7d64c4     273
92686092-27d0-404a-bb67-90da9a9cba66       1
Name: count, dtype: int64

In [15]:
appointments['ServiceId']

0       1efe2465-4741-48c8-8408-114818cdce74
1       1efe2465-4741-48c8-8408-114818cdce74
2       1efe2465-4741-48c8-8408-114818cdce74
3       1efe2465-4741-48c8-8408-114818cdce74
4       1efe2465-4741-48c8-8408-114818cdce74
                        ...                 
2751    1efe2465-4741-48c8-8408-114818cdce74
2752    1efe2465-4741-48c8-8408-114818cdce74
2753    1efe2465-4741-48c8-8408-114818cdce74
2754    1efe2465-4741-48c8-8408-114818cdce74
2755    1efe2465-4741-48c8-8408-114818cdce74
Name: ServiceId, Length: 2756, dtype: object